In [1]:
# pip install opencv-python

  Obtaining dependency information for opencv-python from https://files.pythonhosted.org/packages/c7/ec/9dabb6a9abfdebb3c45b0cc52dec901caafef2b2c7e7d6a839ed86d81e91/opencv_python-4.9.0.80-cp37-abi3-win_amd64.whl.metadata
   ---------------------------------------- 0.0/38.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/38.6 MB 640.0 kB/s eta 0:01:01
    --------------------------------------- 0.7/38.6 MB 11.5 MB/s eta 0:00:04
   --- ------------------------------------ 3.9/38.6 MB 35.0 MB/s eta 0:00:01
   -------- ------------------------------- 7.9/38.6 MB 55.9 MB/s eta 0:00:01
   ------------ --------------------------- 12.0/38.6 MB 131.2 MB/s eta 0:00:01
   ------------- -------------------------- 12.9/38.6 MB 93.9 MB/s eta 0:00:01
   ------------- -------------------------- 13.0/38.6 MB 72.6 MB/s eta 0:00:01
   ------------- -------------------------- 13.2/38.6 MB 54.4 MB/s eta 0:00:01
   -------------- ------------------------- 13.5/38.6 MB 46.7 MB/s eta 0:00:01


# 라이브러리 연결

In [2]:
## 데이터 분석 라이브러리
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import csv
import re
import time
import random
from tqdm.notebook import tqdm

## Selenium 관련 환경설정
from selenium import webdriver as wb
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
# from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import NoSuchElementException, ElementNotInteractableException
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
caps = DesiredCapabilities.CHROME
caps["pageLoadStrategy"] = "none"

## bs 라이브러리
from bs4 import BeautifulSoup
import requests

# 페이지 로드 간소화 라이브러리
from selenium.webdriver.support.ui import WebDriverWait as wait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC

## webdriver options
options = wb.ChromeOptions()
# options.add_argument('--headless')
# options.add_argument('--no-sandbox')
# options.add_argument('disable-dev-shm-usage')

## 이미지 x
# options.add_argument('--disable-images')
# options.add_experimental_option("prefs", {'profile.managed_default_content_settings.images': 2})
# options.add_argument('--blink-settings=imagesEnabled=false')

# yolo 다운로드

In [3]:
import requests

# yolov3.cfg 다운로드
cfg_url = 'https://raw.githubusercontent.com/pjreddie/darknet/master/cfg/yolov3.cfg'
cfg_response = requests.get(cfg_url)
with open('yolov3.cfg', 'wb') as file:
    file.write(cfg_response.content)

# yolov3.weights 다운로드 (파일이 매우 큼)
weights_url = 'https://pjreddie.com/media/files/yolov3.weights'
weights_response = requests.get(weights_url)
with open('yolov3.weights', 'wb') as file:
    file.write(weights_response.content)

# coco.names 다운로드
names_url = 'https://raw.githubusercontent.com/pjreddie/darknet/master/data/coco.names'
names_response = requests.get(names_url)
with open('coco.names', 'wb') as file:
    file.write(names_response.content)

# 데이터 가져오기

In [74]:
data = pd.read_csv('influ_data.csv')
data

,이미지,제목,설명,가수,곡명
0,https://img.youtube.com/vi/Cnh7exHmJ_U/maxresd...,사랑이 머문 자리에,#팝송추천 #플레이리스트 #노래추천 우리가 함께했던 수많은 공간에는 아직도 그때의 ...,Stephen Sanchez,Until I Found You
1,https://img.youtube.com/vi/Cnh7exHmJ_U/maxresd...,사랑이 머문 자리에,#팝송추천 #플레이리스트 #노래추천 우리가 함께했던 수많은 공간에는 아직도 그때의 ...,"MAX, keshi",IT'S YOU
2,https://img.youtube.com/vi/Cnh7exHmJ_U/maxresd...,사랑이 머문 자리에,#팝송추천 #플레이리스트 #노래추천 우리가 함께했던 수많은 공간에는 아직도 그때의 ...,keshi,UNDERSTAND
3,https://img.youtube.com/vi/Cnh7exHmJ_U/maxresd...,사랑이 머문 자리에,#팝송추천 #플레이리스트 #노래추천 우리가 함께했던 수많은 공간에는 아직도 그때의 ...,Arash Buana,stars
4,https://img.youtube.com/vi/Cnh7exHmJ_U/maxresd...,사랑이 머문 자리에,#팝송추천 #플레이리스트 #노래추천 우리가 함께했던 수많은 공간에는 아직도 그때의 ...,"Jesse Barrera, Nieman, Melissa Polinar",Tangled Up (Reimagined)
...,...,...,...,...,...
2296,https://img.youtube.com/vi/1QEUJ5d7WO8/maxresd...,나만 알고 싶은 새벽감성 사클 노래 모음,#노래추천 #음악추천 #사운드클라우드 구독자 1000명! 다들 너무 고마워요!그래서...,Bas Bao,"넌 없고, 난 머물고"
2297,https://img.youtube.com/vi/1QEUJ5d7WO8/maxresd...,나만 알고 싶은 새벽감성 사클 노래 모음,#노래추천 #음악추천 #사운드클라우드 구독자 1000명! 다들 너무 고마워요!그래서...,D'sperado,"O/W (with hatts, HYE SUNG)"
2298,https://img.youtube.com/vi/1QEUJ5d7WO8/maxresd...,나만 알고 싶은 새벽감성 사클 노래 모음,#노래추천 #음악추천 #사운드클라우드 구독자 1000명! 다들 너무 고마워요!그래서...,DAWN,불면증 (白夜) (feat. YAYYOUNG) (prod. by N O V E L)
2299,https://img.youtube.com/vi/1QEUJ5d7WO8/maxresd...,나만 알고 싶은 새벽감성 사클 노래 모음,#노래추천 #음악추천 #사운드클라우드 구독자 1000명! 다들 너무 고마워요!그래서...,Mabinc,혼잣말 (feat. Ripley)


# 이미지 링크를 통하여 객체 인식

In [75]:
import cv2
import numpy as np
import requests

# YOLO 모델 로드
net = cv2.dnn.readNet("yolov3.weights", "yolov3.cfg")
with open("coco.names", "r") as f:
    classes = [line.strip() for line in f.readlines()]

layer_names = net.getLayerNames()

# OpenCV의 버전이나 특정 구현에 따라 반환 형태가 다를 수 있으므로,
# 반환된 값이 리스트의 리스트(2차원 배열)인지 확인하고, 그렇다면 첫 번째 요소를 사용합니다.
# 그렇지 않으면 바로 인덱스로 사용합니다.
unconnected_out_layers = net.getUnconnectedOutLayers()
if len(unconnected_out_layers.shape) == 2:
    output_layers = [layer_names[i[0] - 1] for i in unconnected_out_layers]
else:
    output_layers = [layer_names[i - 1] for i in unconnected_out_layers]

# 이미지 URL에서 직접 이미지를 메모리에 로드하는 함수
def load_image_from_url(url):
    response = requests.get(url)
    image_array = np.asarray(bytearray(response.content), dtype=np.uint8)
    image = cv2.imdecode(image_array, cv2.IMREAD_COLOR)
    return image

# 객체 탐지 함수
def detect_objects(img):
    height, width, channels = img.shape
    blob = cv2.dnn.blobFromImage(img, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
    net.setInput(blob)
    outs = net.forward(output_layers)
    class_ids = []
    confidences = []
    boxes = []
    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.5:
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)
                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)
    indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)
    highest_confidence_index = -1
    highest_confidence = 0
    detected_label = ""
    for i in range(len(boxes)):
        if i in indexes:
            x, y, w, h = boxes[i]
            label = str(classes[class_ids[i]])
            confidence = confidences[i]
            if confidence > highest_confidence:
                highest_confidence = confidence
                highest_confidence_index = i
                detected_label = label  # 신뢰도 없이 객체 이름만 가져오기
            color = (0, 255, 0)
            cv2.rectangle(img, (x, y), (x + w, y + h), color, 2)
            cv2.putText(img, f"{label}: {confidence:.2f}", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
    return img, detected_label

In [76]:
# 여러 이미지 URL 리스트
image_urls = data['이미지'].unique()
imgobj_list = []
# 각 이미지에 대해 객체 탐지 수행
for image_url in tqdm(image_urls):
    image = load_image_from_url(image_url)
    detected_image, highest_confidence_label = detect_objects(image)
    imgobj_list.append(highest_confidence_label)

  0%|          | 0/133 [00:00<?, ?it/s]

In [77]:
# 이미지 객체 컬럼 생성
data['이미지객체']=data['이미지']
# 이미지 객체 전처리
img_list = image_urls.tolist()
imgobj_dic = dict(zip(img_list, imgobj_list))
data['이미지객체'] = data['이미지객체'].map(imgobj_dic)
data

,이미지,제목,설명,가수,곡명,이미지객체
0,https://img.youtube.com/vi/Cnh7exHmJ_U/maxresd...,사랑이 머문 자리에,#팝송추천 #플레이리스트 #노래추천 우리가 함께했던 수많은 공간에는 아직도 그때의 ...,Stephen Sanchez,Until I Found You,person
1,https://img.youtube.com/vi/Cnh7exHmJ_U/maxresd...,사랑이 머문 자리에,#팝송추천 #플레이리스트 #노래추천 우리가 함께했던 수많은 공간에는 아직도 그때의 ...,"MAX, keshi",IT'S YOU,person
2,https://img.youtube.com/vi/Cnh7exHmJ_U/maxresd...,사랑이 머문 자리에,#팝송추천 #플레이리스트 #노래추천 우리가 함께했던 수많은 공간에는 아직도 그때의 ...,keshi,UNDERSTAND,person
3,https://img.youtube.com/vi/Cnh7exHmJ_U/maxresd...,사랑이 머문 자리에,#팝송추천 #플레이리스트 #노래추천 우리가 함께했던 수많은 공간에는 아직도 그때의 ...,Arash Buana,stars,person
4,https://img.youtube.com/vi/Cnh7exHmJ_U/maxresd...,사랑이 머문 자리에,#팝송추천 #플레이리스트 #노래추천 우리가 함께했던 수많은 공간에는 아직도 그때의 ...,"Jesse Barrera, Nieman, Melissa Polinar",Tangled Up (Reimagined),person
...,...,...,...,...,...,...
2296,https://img.youtube.com/vi/1QEUJ5d7WO8/maxresd...,나만 알고 싶은 새벽감성 사클 노래 모음,#노래추천 #음악추천 #사운드클라우드 구독자 1000명! 다들 너무 고마워요!그래서...,Bas Bao,"넌 없고, 난 머물고",
2297,https://img.youtube.com/vi/1QEUJ5d7WO8/maxresd...,나만 알고 싶은 새벽감성 사클 노래 모음,#노래추천 #음악추천 #사운드클라우드 구독자 1000명! 다들 너무 고마워요!그래서...,D'sperado,"O/W (with hatts, HYE SUNG)",
2298,https://img.youtube.com/vi/1QEUJ5d7WO8/maxresd...,나만 알고 싶은 새벽감성 사클 노래 모음,#노래추천 #음악추천 #사운드클라우드 구독자 1000명! 다들 너무 고마워요!그래서...,DAWN,불면증 (白夜) (feat. YAYYOUNG) (prod. by N O V E L),
2299,https://img.youtube.com/vi/1QEUJ5d7WO8/maxresd...,나만 알고 싶은 새벽감성 사클 노래 모음,#노래추천 #음악추천 #사운드클라우드 구독자 1000명! 다들 너무 고마워요!그래서...,Mabinc,혼잣말 (feat. Ripley),


In [70]:
from selenium import webdriver as wb
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
import cv2
import numpy as np
from collections import Counter
import requests
from io import BytesIO
from PIL import Image

def extract_colors(image_cv2, num_colors=3):
    # 주요 색상 추출 함수 (상위 3가지 색상)
    pixels = image_cv2.reshape(-1, 3)
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 200, .1)
    flags = cv2.KMEANS_RANDOM_CENTERS
    _, labels, palette = cv2.kmeans(pixels.astype(np.float32), num_colors, None, criteria, 10, flags)
    counts = Counter(labels.flatten())
    
    dominant_colors = []
    for index in counts.most_common(num_colors):
        dominant_colors.append(palette[index[0]].astype(int))
    
    return dominant_colors

def get_all_colors(image_urls):
    all_colors_list = []  # 모든 색상을 담을 리스트
    for url in tqdm(image_urls):
        try:
            image = download_image(url)
            image_cv2 = cv2.cvtColor(np.array(image), cv2.COLOR_RGB2BGR)
            dominant_colors = extract_colors(image_cv2, num_colors=3)
            # 이미지에서 추출된 각 색상을 개별적으로 리스트에 추가
            all_colors_list.extend(dominant_colors)
            # print(dominant_colors)
            # print("리스트", all_colors_list)
        except Exception as e:
            print(f"Error processing image at {url}: {e}")
            # 오류가 발생한 경우, 해당 이미지에 대한 색상은 추가하지 않음
    return all_colors_list

In [66]:
# 이미지 URL 목록
image_urls = data['이미지']
all_colors = get_all_colors(image_urls)

[array([ 7, 10, 13]), array([ 83, 138, 161]), array([18, 50, 83])]
리스트 [array([ 7, 10, 13]), array([ 83, 138, 161]), array([18, 50, 83])]
[array([ 7, 10, 13]), array([ 83, 138, 161]), array([18, 50, 83])]
리스트 [array([ 7, 10, 13]), array([ 83, 138, 161]), array([18, 50, 83]), array([ 7, 10, 13]), array([ 83, 138, 161]), array([18, 50, 83])]
[array([ 7, 10, 13]), array([ 83, 138, 161]), array([18, 50, 83])]
리스트 [array([ 7, 10, 13]), array([ 83, 138, 161]), array([18, 50, 83]), array([ 7, 10, 13]), array([ 83, 138, 161]), array([18, 50, 83]), array([ 7, 10, 13]), array([ 83, 138, 161]), array([18, 50, 83])]
[array([ 7, 10, 13]), array([ 83, 138, 161]), array([18, 50, 83])]
리스트 [array([ 7, 10, 13]), array([ 83, 138, 161]), array([18, 50, 83]), array([ 7, 10, 13]), array([ 83, 138, 161]), array([18, 50, 83]), array([ 7, 10, 13]), array([ 83, 138, 161]), array([18, 50, 83]), array([ 7, 10, 13]), array([ 83, 138, 161]), array([18, 50, 83])]
[array([ 7, 10, 13]), array([ 83, 138, 161]), array(

KeyboardInterrupt: 

In [78]:
from selenium import webdriver as wb
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
import cv2
import numpy as np
from collections import Counter
import requests
from io import BytesIO
from PIL import Image

def extract_colors(image_cv2, num_colors=3):
    # 주요 색상 추출 함수 (상위 3가지 색상 및 각 색상이 차지하는 비율)
    pixels = image_cv2.reshape(-1, 3)
    num_pixels = len(pixels)
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 200, .1)
    flags = cv2.KMEANS_RANDOM_CENTERS
    _, labels, palette = cv2.kmeans(pixels.astype(np.float32), num_colors, None, criteria, 10, flags)
    counts = Counter(labels.flatten())

    dominant_colors = []
    for index in counts.most_common(num_colors):
        color_percentage = (index[1] / num_pixels) * 100 # 색상의 비율 계산
        color_info = {
            "color": palette[index[0]].astype(int), 
            "percentage": color_percentage
        }
        dominant_colors.append(color_info)
    
    return dominant_colors


def get_all_colors(image_urls):
    all_colors_list = []  # 모든 색상을 담을 리스트
    for url in tqdm(image_urls):
        try:
            image = download_image(url)
            image_cv2 = cv2.cvtColor(np.array(image), cv2.COLOR_RGB2BGR)
            dominant_colors = extract_colors(image_cv2, num_colors=3)
            # 이미지에서 추출된 각 색상을 개별적으로 리스트에 추가
            all_colors_list.extend(dominant_colors)
            # print(dominant_colors)
            # print(all_colors_list)
        except Exception as e:
            print(f"Error processing image at {url}: {e}")
            # 오류가 발생한 경우, 해당 이미지에 대한 색상은 추가하지 않음
    return all_colors_list

In [80]:
# 이미지 URL 목록
image_urls = data['이미지'].unique()
all_colors = get_all_colors(image_urls)

  0%|          | 0/133 [00:00<?, ?it/s]

In [81]:
# 이미지 객체 컬럼 생성
data['이미지색감']=data['이미지']
# 이미지 객체 전처리
img_list = image_urls.tolist()
imgcolor_dic = dict(zip(img_list, all_colors))
data['이미지색감'] = data['이미지색감'].map(imgcolor_dic)
data

,이미지,제목,설명,가수,곡명,이미지객체,이미지색감
0,https://img.youtube.com/vi/Cnh7exHmJ_U/maxresd...,사랑이 머문 자리에,#팝송추천 #플레이리스트 #노래추천 우리가 함께했던 수많은 공간에는 아직도 그때의 ...,Stephen Sanchez,Until I Found You,person,"{'color': [7, 10, 13], 'percentage': 53.565972..."
1,https://img.youtube.com/vi/Cnh7exHmJ_U/maxresd...,사랑이 머문 자리에,#팝송추천 #플레이리스트 #노래추천 우리가 함께했던 수많은 공간에는 아직도 그때의 ...,"MAX, keshi",IT'S YOU,person,"{'color': [7, 10, 13], 'percentage': 53.565972..."
2,https://img.youtube.com/vi/Cnh7exHmJ_U/maxresd...,사랑이 머문 자리에,#팝송추천 #플레이리스트 #노래추천 우리가 함께했던 수많은 공간에는 아직도 그때의 ...,keshi,UNDERSTAND,person,"{'color': [7, 10, 13], 'percentage': 53.565972..."
3,https://img.youtube.com/vi/Cnh7exHmJ_U/maxresd...,사랑이 머문 자리에,#팝송추천 #플레이리스트 #노래추천 우리가 함께했던 수많은 공간에는 아직도 그때의 ...,Arash Buana,stars,person,"{'color': [7, 10, 13], 'percentage': 53.565972..."
4,https://img.youtube.com/vi/Cnh7exHmJ_U/maxresd...,사랑이 머문 자리에,#팝송추천 #플레이리스트 #노래추천 우리가 함께했던 수많은 공간에는 아직도 그때의 ...,"Jesse Barrera, Nieman, Melissa Polinar",Tangled Up (Reimagined),person,"{'color': [7, 10, 13], 'percentage': 53.565972..."
...,...,...,...,...,...,...,...
2296,https://img.youtube.com/vi/1QEUJ5d7WO8/maxresd...,나만 알고 싶은 새벽감성 사클 노래 모음,#노래추천 #음악추천 #사운드클라우드 구독자 1000명! 다들 너무 고마워요!그래서...,Bas Bao,"넌 없고, 난 머물고",,"{'color': [26, 25, 32], 'percentage': 48.61773..."
2297,https://img.youtube.com/vi/1QEUJ5d7WO8/maxresd...,나만 알고 싶은 새벽감성 사클 노래 모음,#노래추천 #음악추천 #사운드클라우드 구독자 1000명! 다들 너무 고마워요!그래서...,D'sperado,"O/W (with hatts, HYE SUNG)",,"{'color': [26, 25, 32], 'percentage': 48.61773..."
2298,https://img.youtube.com/vi/1QEUJ5d7WO8/maxresd...,나만 알고 싶은 새벽감성 사클 노래 모음,#노래추천 #음악추천 #사운드클라우드 구독자 1000명! 다들 너무 고마워요!그래서...,DAWN,불면증 (白夜) (feat. YAYYOUNG) (prod. by N O V E L),,"{'color': [26, 25, 32], 'percentage': 48.61773..."
2299,https://img.youtube.com/vi/1QEUJ5d7WO8/maxresd...,나만 알고 싶은 새벽감성 사클 노래 모음,#노래추천 #음악추천 #사운드클라우드 구독자 1000명! 다들 너무 고마워요!그래서...,Mabinc,혼잣말 (feat. Ripley),,"{'color': [26, 25, 32], 'percentage': 48.61773..."


In [82]:
# 데이터프레임의 각 행을 3번씩 반복하여 새로운 데이터프레임 생성
data = pd.DataFrame(
    data.values.repeat(3, axis=0), # 값들을 3번 반복
    columns=data.columns) # 원본 데이터프레임의 컬럼명 사용

In [83]:
data

,이미지,제목,설명,가수,곡명,이미지객체,이미지색감
0,https://img.youtube.com/vi/Cnh7exHmJ_U/maxresd...,사랑이 머문 자리에,#팝송추천 #플레이리스트 #노래추천 우리가 함께했던 수많은 공간에는 아직도 그때의 ...,Stephen Sanchez,Until I Found You,person,"{'color': [7, 10, 13], 'percentage': 53.565972..."
1,https://img.youtube.com/vi/Cnh7exHmJ_U/maxresd...,사랑이 머문 자리에,#팝송추천 #플레이리스트 #노래추천 우리가 함께했던 수많은 공간에는 아직도 그때의 ...,Stephen Sanchez,Until I Found You,person,"{'color': [7, 10, 13], 'percentage': 53.565972..."
2,https://img.youtube.com/vi/Cnh7exHmJ_U/maxresd...,사랑이 머문 자리에,#팝송추천 #플레이리스트 #노래추천 우리가 함께했던 수많은 공간에는 아직도 그때의 ...,Stephen Sanchez,Until I Found You,person,"{'color': [7, 10, 13], 'percentage': 53.565972..."
3,https://img.youtube.com/vi/Cnh7exHmJ_U/maxresd...,사랑이 머문 자리에,#팝송추천 #플레이리스트 #노래추천 우리가 함께했던 수많은 공간에는 아직도 그때의 ...,"MAX, keshi",IT'S YOU,person,"{'color': [7, 10, 13], 'percentage': 53.565972..."
4,https://img.youtube.com/vi/Cnh7exHmJ_U/maxresd...,사랑이 머문 자리에,#팝송추천 #플레이리스트 #노래추천 우리가 함께했던 수많은 공간에는 아직도 그때의 ...,"MAX, keshi",IT'S YOU,person,"{'color': [7, 10, 13], 'percentage': 53.565972..."
...,...,...,...,...,...,...,...
6898,https://img.youtube.com/vi/1QEUJ5d7WO8/maxresd...,나만 알고 싶은 새벽감성 사클 노래 모음,#노래추천 #음악추천 #사운드클라우드 구독자 1000명! 다들 너무 고마워요!그래서...,Mabinc,혼잣말 (feat. Ripley),,"{'color': [26, 25, 32], 'percentage': 48.61773..."
6899,https://img.youtube.com/vi/1QEUJ5d7WO8/maxresd...,나만 알고 싶은 새벽감성 사클 노래 모음,#노래추천 #음악추천 #사운드클라우드 구독자 1000명! 다들 너무 고마워요!그래서...,Mabinc,혼잣말 (feat. Ripley),,"{'color': [26, 25, 32], 'percentage': 48.61773..."
6900,https://img.youtube.com/vi/1QEUJ5d7WO8/maxresd...,나만 알고 싶은 새벽감성 사클 노래 모음,#노래추천 #음악추천 #사운드클라우드 구독자 1000명! 다들 너무 고마워요!그래서...,YAYYOUNG,do you feel like you have no one to turn to?,,"{'color': [26, 25, 32], 'percentage': 48.61773..."
6901,https://img.youtube.com/vi/1QEUJ5d7WO8/maxresd...,나만 알고 싶은 새벽감성 사클 노래 모음,#노래추천 #음악추천 #사운드클라우드 구독자 1000명! 다들 너무 고마워요!그래서...,YAYYOUNG,do you feel like you have no one to turn to?,,"{'color': [26, 25, 32], 'percentage': 48.61773..."


In [84]:
data.to_csv("influ_full_data.csv", index=False, encoding="utf-8-sig")

In [85]:
data.index.name = '번호'
data.to_csv("influ__full_data_index.csv", index=False, encoding="utf-8-sig")